In [1]:
import re
import faiss
import numpy as np
import pandas as pd
from collections import Counter
import networkx as nx
from sentence_transformers import SentenceTransformer

In [2]:
df_vacancies = pd.read_parquet('../data_artefacts')

In [3]:
df_vacancies

,vacancy_id,title,city,keywords,skills,company,industry,experience,salary_from,salary_to,salary_currency,salary_str,url,published_at,source_vacancy
0,124523955,Менеджер продукта,Москва,Ты любишь общаться с людьми и умеешь находить ...,[],Дальневосточный банк,Банк,От 1 года до 3 лет,80000.0,NaN,RUR,80000- RUR,https://hh.ru/vacancy/124523955,2025-08-27 05:28:36+00:00,Product Manager
1,124826328,Delivery Manager / Деливери менеджер,Москва,Опыт управления ИТ-проектами от 5 лет (планиро...,[],Брайт Эйдженси,Кадровые агентства,От 3 до 6 лет,NaN,NaN,None,з/п не указана,https://hh.ru/vacancy/124826328,2025-09-03 11:34:11+00:00,Machine Learning
2,124964211,DevOps-инженер в команду User Storage & Policy...,Москва,Понимание основ сетевых технологий (стек TCP/I...,[],WILDBERRIES,Интернет-магазин,От 3 до 6 лет,NaN,NaN,None,з/п не указана,https://hh.ru/vacancy/124964211,2025-09-05 16:00:35+00:00,Python Developer
3,124374678,Системный аналитик/Аналитик данных ML,Москва,"Знание основ http-запросов, опыт работы с <hig...",[],Центр Информационных технологий Роскадастр-Инф...,Unknown,От 1 года до 3 лет,170000.0,NaN,RUR,170000- RUR,https://hh.ru/vacancy/124374678,2025-08-22 11:36:06+00:00,Web Analyst
4,123978202,PowerPlatform Engineer,Москва,"Strong skills in algorithms, <highlighttext>da...","[Power BI, MS SharePoint, MS SQL Server, Azure...",ACCESA,"Системная интеграция, автоматизации технологи...",От 3 до 6 лет,NaN,NaN,None,з/п не указана,https://hh.ru/vacancy/123978202,2025-08-12 14:03:02+00:00,Data Engineer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17571,124789341,Преподаватель курса Backend-разработка (разраб...,Москва,Знание или опыт работы как минимум в одном из ...,"[Python, C++, С#, ADO.NET, ООП, SQL, UML, Архи...",Компьютерная академия TOP (Компьютерная Академ...,"Вуз, ссуз колледж, ПТУ",От 1 года до 3 лет,50000.0,NaN,RUR,50000- RUR,https://hh.ru/vacancy/124789341,2025-09-02 15:47:05+00:00,Python Developer
17572,123899058,Оператор по обработке заявок,Москва,Имеете базовые навыки работы с ПК (Google Docs...,[],Ирбис Аналитика,Разработка программного обеспечения,Нет опыта,40000.0,60000.0,RUR,40000-60000 RUR,https://hh.ru/vacancy/123899058,2025-09-07 10:24:25+00:00,Аналитик данных
17573,125035681,Менеджер проекта (строительство),Москва,Опыт работы от 1 года в аналогичной должности ...,"[Деловая переписка, Организаторские навыки, Де...",Кипарис МСК,Unknown,От 1 года до 3 лет,75000.0,75000.0,RUR,75000-75000 RUR,https://hh.ru/vacancy/125035681,2025-09-08 11:47:24+00:00,Project Manager
17574,124426292,DataOps/DevOps инженер на DataLake,Москва,Опыта работы по направлению DataOps/DevOps/<hi...,"[Linux, Kubernetes, S3, Spark, Impala, Hive, T...",Ингосстрах,"Страхование, перестрахование",От 3 до 6 лет,NaN,NaN,None,з/п не указана,https://hh.ru/vacancy/124426292,2025-08-25 08:06:50+00:00,Data Engineer


In [4]:
model = SentenceTransformer('efederici/sentence-bert-base')

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r"[^a-zа-я0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text

vacancy_texts = [
    normalize_text(
        f"{row['title']} {row['company']} {', '.join(row['skills'])} {row['experience']} {row['keywords']}"
    )
    for _, row in df_vacancies.iterrows()
]

vacancy_embeddings = model.encode(vacancy_texts, convert_to_numpy=True).astype("float32")
vacancy_embeddings /= np.linalg.norm(vacancy_embeddings, axis=1, keepdims=True)

In [6]:
# FAISS индекс
dim = vacancy_embeddings.shape[1]
faiss_index = faiss.IndexHNSWFlat(dim, 32)  # HNSW для быстрого поиска
faiss_index.add(vacancy_embeddings)

In [7]:
G = nx.DiGraph()
position_nodes = []

for i, row in df_vacancies.iterrows():
    pos_node = row["title"]
    position_nodes.append(pos_node)
    
    # Вершины вакансии
    G.add_node(pos_node, type="position", vacancy_id=row["vacancy_id"], 
               company=row["company"], experience=row["experience"],
               salary=row["salary_str"], industry=row["industry"],
               embedding=vacancy_embeddings[i])
    
    if row["company"]:
        G.add_node(row["company"], type="company")
        G.add_edge(pos_node, row["company"])
    if row["experience"]:
        G.add_node(row["experience"], type="level")
        G.add_edge(pos_node, row["experience"])
    if row["industry"]:
        G.add_node(row["industry"], type="domain")
        G.add_edge(pos_node, row["industry"])

    # Навыки
    skills_arr = row["skills"]
    if isinstance(skills_arr, (list, np.ndarray)):
        for skill in skills_arr:
            skill = str(skill).strip()
            if skill:
                G.add_node(skill, type="skill")
                G.add_edge(pos_node, skill)  # position → skill
                G.add_edge(skill, pos_node)  # skill → position

In [9]:
def recommend_vacancies(user_text, top_k=5, top_career=1, min_skill_freq=2, top_skills=10):
    user_vector = model.encode([normalize_text(user_text)], convert_to_numpy=True).astype("float32")
    user_vector /= np.linalg.norm(user_vector, axis=1, keepdims=True)
    # Поиск топ-K вакансий по FAISS
    distances, indices = faiss_index.search(user_vector, top_k)
    
    recommendations = []
    career_paths = set()
    all_neighbor_skills = []
    
    for idx in indices[0]:
        node = position_nodes[idx]
        n_data = G.nodes[node]

        skills = [s for s in G.successors(node) if G.nodes[s]["type"]=="skill"]
        
        recommendations.append({
            "title": node,
            "company": n_data["company"],
            "experience": n_data["experience"],
            "salary": n_data["salary"],
            "industry": n_data["industry"],
            "skills": skills
        })
        
        # Топ-N похожих позиций через навыки
        for skill in skills:
            neighbors = [pos for pos in G.successors(skill) if G.nodes[pos]["type"]=="position" and pos != node]
            if neighbors:
                # Сортируем по косинусной близости эмбеды
                neighbor_embeddings = np.array([G.nodes[p]["embedding"] for p in neighbors]).astype("float32")
                sims = neighbor_embeddings @ user_vector.T
                # Берем только top_career ближайших соседей
                top_idx = np.argsort(-sims.ravel())[:top_career]
                for i in top_idx:
                    neighbor_pos = neighbors[i]
                    career_paths.add(neighbor_pos)
                    
                    # Вытаскиваем навыки у соседа
                    neighbor_skills = [
                        s for s in G.successors(neighbor_pos) if G.nodes[s]["type"] == "skill"
                    ]
                    all_neighbor_skills.extend(neighbor_skills)
    
    # Фильтрация навыков по частоте
    skill_counts = Counter(all_neighbor_skills)
    filtered_skills = [s for s, c in skill_counts.items() if c >= min_skill_freq]
    
    # Ограничиваем топ N по частоте
    expanded_skills = [s for s, _ in skill_counts.most_common(top_skills) if s in filtered_skills]

    return recommendations, expanded_skills, list(career_paths)

user_query = "ML engineer с опытом PyTorch 1 год"
recs, skills, paths = recommend_vacancies(user_query)

print("Рекомендованные вакансии (ближайшие к user vector через faiss:")
for r in recs:
    print(r)

print("\nНавыки для апгрейда (через граф):")
print(skills)

print("\nПотенциальные карьерные переходы:")
print(paths)

Рекомендованные вакансии (ближайшие к user vector через faiss:
{'title': 'ML Engineer', 'company': 'Эвотор', 'experience': 'От 3 до 6 лет', 'salary': 'з/п не указана', 'industry': 'Разработка программного обеспечения', 'skills': ['Python', 'PyTorch', 'Linux', 'Git', 'Computer Vision', 'MLOps', 'Machine Learning', 'ML', 'Математическая статистика', 'Алгоритмы и структуры данных', 'SQL', 'Машинное обучение', 'Data Science', 'NLP', 'Английский язык', 'MySQL', 'C/C++', 'NLU', 'Clickhouse', 'Теория вероятностей', 'Классическое машинное обучение', 'A/B тесты', 'PySpark', 'TensorFlow', 'Базы данных', 'n8n', 'AI', 'Deep Learning', 'Docker', 'ООП', 'LLM', 'RAG', 'Langchain', 'FastAPI']}
{'title': 'Data scientist', 'company': 'Страховая компания Freedom Insurance', 'experience': 'От 3 до 6 лет', 'salary': 'з/п не указана', 'industry': 'Страхование, перестрахование', 'skills': ['Python', 'pandas', 'Numpy', 'Анализ временных рядов', 'Машинное обучение', 'ML', 'SQL', 'Анализ данных', 'Базы данных']